# Holistic Path Benchmark
In this experiment we will try to produce comparable results between various learning methods. We will use Holistic Path as our task and train a centralised, federated, swarm, and federated baseline model. We will then compare the results of these models to see how decentralised learning compares to centralised learning for this specific task.

Our configuration for this specific setup is as follows:
- Balanced Dataset
- 100% of the balanced dataset is used
- ROI frame cropping
- Pre-trained ResNet18
- 5 Clients
- Swarm setup is a fully connected static network

Below is the full configuration


In [17]:
import json

config = {
    "data": {
        "path": "/mnt/ZOD",
        "version": "full",
        "ratio": 0.05,
        "ratio_val": 1,
        "skip_test": True,
        "img_size": [256, 256],
        "transforms": "[CustomCrop(800, 500, 800, 2840), Resize(img_size), Normalize(mean=[0.337, 0.345, 0.367], std=[0.160, 0.180, 0.214])]",
        "shuffle_seed": 101,
        "decentralised_shuffle_seed": 101,
        "dataloader_args": {
            "num_workers": 8,
            "prefetch_factor": 128,
            "batch_size": 16,
            "pin_memory": False,
            "shuffle": True,
        },
        "train_val_id_generator": "balanced_frames_kilichenko",
        "dataset_getitem": "interpolated_target_distances"
    },
    "model": {
        "name": "mobilenet",
        "args": {
            "num_output": 51
        },
        "loss": "L1Loss",
        "optimiser": "Adam",
        "optimiser_args": {
            "lr": 1e-3
        }
    },
    "central": {
        "train": False,
        "use_gpu": True,
        "epochs": 150
    }, 
}

with open("config.json", "w") as f:
    json.dump(config, f, indent=4)

In [18]:
from copy import deepcopy

decentralised_config = deepcopy(config)

decentralised = {
        "train": ["baseline"],
        "global":{
            "n_clients":5,
            "global_rounds":10,
            "client_resources":{
                "num_cpus": 1, 
                "num_gpus": 0.19
            },
            "ray_init_args":{
                "include_dashboard": True,
                "num_cpus": 7,
                "num_gpus": 1
            },
            "swarm_orchestrator": "synchronous_fixed_rounds_fc",
        },
        "client": {
            "epochs": 10
        }
    }

decentralised_config["decentralised"] = decentralised
decentralised_config["data"]["dataloader_args"]["num_workers"] = 2

with open("decentralised_config.json", "w") as f:
    json.dump(decentralised_config, f, indent=4)

In [14]:
from fedswarm.utils.training.utils import get_best_device
a = get_best_device()